## 参考链接
[Sinusoidal位置编码](https://spaces.ac.cn/archives/8231 "苏剑林")

[位置编码的理解](https://spaces.ac.cn/archives/10347 "苏剑林")

[让研究人员绞尽脑汁的Transformer位置编码](https://spaces.ac.cn/archives/8130 "苏剑林")

[RoPE位置编码](https://spaces.ac.cn/archives/8265 "苏剑林，该方法提出者")

## 位置编码的作用
### 1. **破环Attention的置换不变性**
置换不变性是指对于双向attetion而言，$f(...,x_m,...,x_n) = f(...,x_n,...,x_m)$,及个体在整体中的不同位置具有相同的效果，显然不符合语言常识。

### 2. 添加先验知识
位置信息可以作为attention的一种先验知识。
sinusoidal位置编码使用三角函数生成的绝对位置编码，并且相邻的两个向量相似度更高，隐含了相近的token应该具有相近的Embedding的先验。Bert也是绝对位置编码
相对位置编码（主流）和诸如RNN、CNN等模型也自然的包含先验知识，越近的token越重要
虽然不需要位置编码（NoPE+Cross Attention）也取得了比较的好的结果，crossattention是指单边注意力机制，只能看到自己和之前的信息，这种注意力本身就不具备置换不变性。这种单向注意力机制其方差也包含了位置信息。但实际上只是说这种方法能够取得跟加位置编码相似的结果，无法证明其优越性。

在苏神的文章中提出：越少的先验信息，代表人为的偏见和误区更少，从而天花板更高。这个观点很有趣

## 位置编码的目标
1. 位置信息：绝对位置信息和相对位置信息
2. 远程衰减：距离越远，相关性应该适当衰减

## 位置编码方式
* 绝对位置编码：sinusoidal位置编码，transformer论文中提到的那种编码方式
* 相对位置编码：RoPE位置编码，主流的编码方法

In [38]:
import torch
from torch import nn
import math

class SinusoidalPositional(nn.Module):
    def __init__(self, d_model: int, num_heads: int, max_len: int = 5000):
        super().__init__()
        # 计算每个头的维度
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        head_dim = d_model // num_heads
        pe = torch.zeros((max_len, head_dim))
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(dim=1)
        dim = torch.exp(- torch.arange(0, head_dim, 2) * torch.log(torch.tensor(10000.0)) / head_dim)
        pe[:, 0::2] = torch.sin(pos * dim)
        pe[:, 1::2] = torch.cos(pos * dim)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe",pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """sinusoidal位置编码实现

        Args:
            x (torch.Tensor): batch_size, num_heads, seq_len, head_dim

        Returns:
            torch.Tensor: batch_size, num_heads, seq_len, head_dim
        """
        batch_size, num_heads, seq_len, _ = x.size()
        pe = self.pe[:,:seq_len,:].unsqueeze(1).expand(-1, num_heads, -1, -1)
        return x + pe

# 示例使用多头注意力输入
if __name__ == "__main__":
    # 定义参数
    batch_size = 2
    num_heads = 4
    seq_len = 10
    head_dim = 64
    d_model = num_heads * head_dim

    # 创建多头注意力输入张量
    x = torch.randn(batch_size, num_heads, seq_len, head_dim)

    # 创建正弦位置编码模块
    pos_encoder = SinusoidalPositional(d_model, num_heads)

    # 应用位置编码
    x_with_pos = pos_encoder(x)

    print("Original input shape:", x.shape)
    print("Input with position encoding shape:", x_with_pos.shape)

Original input shape: torch.Size([2, 4, 10, 64])
Input with position encoding shape: torch.Size([2, 4, 10, 64])


# RoPE
是一种用绝对位置编码实现相对位置编码的方式。能够适用于线性attention。注意下面是工程实现版本，跟原始论文中的公式存在一点出入，但不影响结果。

In [2]:
import torch
from torch import nn

def rotate_half(x):
    """优化维度注释和负数处理"""
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q, k, freqs):
    """支持多头注意力的版本"""
    # freqs 维度: [1, 1, seq_len, dim]
    # q/k 维度:   [batch, heads, seq_len, dim]
    q_rot = (q * freqs.cos()) + (rotate_half(q) * freqs.sin())
    k_rot = (k * freqs.cos()) + (rotate_half(k) * freqs.sin())
    return q_rot, k_rot

class RotaryPositionEmbedding(nn.Module):
    def __init__(self, dim: int, base: int=10000, max_seq_len: int = 2048):
        super().__init__()
        self.dim = dim
        self.max_seq_len = max_seq_len
        
        # 更准确的变量命名
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

    def forward(self, q, k, seq_len=None):
        """支持多头注意力的维度版本
        Args:
            q: [batch, heads, seq_len, dim]
            k: [batch, heads, seq_len, dim]
        """
        device, dtype = q.device, q.dtype
        seq_len = seq_len if seq_len else q.size(-2)
        
        # 动态生成位置编码
        t = torch.arange(seq_len, device=device, dtype=dtype)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        
        # 维度对齐调整 [batch, heads, seq, dim]
        return apply_rotary_pos_emb(q, k, emb.unsqueeze(0).unsqueeze(0))

        

# 多维度测试案例
def test_rope():
    dim = 4
    # 案例1：基础测试
    x = torch.tensor([[[[1.0, 2.0, 3.0, 4.0]]]])  # [1,1,1,4]
    rope = RotaryPositionEmbedding(dim)
    q_rot, k_rot = rope(x, x)
    print("Case1 Output:", q_rot)

    # 案例2：多头测试
    x_multihead = torch.randn(2, 8, 256, dim)  # [batch=2, heads=8, seq=256, dim=4]
    q_rot, k_rot = rope(x_multihead, x_multihead)
    print("Case2 Output shape:", q_rot.shape)


test_rope()


Case1 Output: tensor([[[[1., 2., 3., 4.]]]])
Case2 Output shape: torch.Size([2, 8, 256, 4])
